In [ ]:
from pathlib import Path

import numpy as np
import torch

from mqt.predictor import ml

source_path = Path(
    "/Users/di93vaw/mqt-predictor/src/mqt/predictor/ml/training_data/training_circuits/training_data_device_selection"
)
target_path = Path("/Users/di93vaw/mqt-predictor/src/mqt/predictor/ml/training_data/training_circuits_compiled")

predictor = ml.Predictor()
figure_of_merit = "expected_fidelity"

# predictor.generate_compiled_circuits(source_path, target_path)
training_data, name_list, scores_list = predictor.generate_trainingdata_from_qasm_files(
    figure_of_merit, path_uncompiled_circuits=source_path
)

ml.helper.save_training_data(training_data, name_list, scores_list, figure_of_merit)
training_data = predictor.get_prepared_training_data(figure_of_merit=figure_of_merit, save_non_zero_indices=True)

X_train = training_data.X_train
X_test = training_data.X_test
y_train = training_data.y_train
y_test = training_data.y_test
indices_train = training_data.indices_train
indices_test = training_data.indices_test
names_list = training_data.names_list
scores_list = training_data.scores_list

scores_filtered = [scores_list[i] for i in indices_test]
names_filtered = [names_list[i] for i in indices_test]

performance = []

In [ ]:
from torch_geometric.data import Dataset


class MyDataset(Dataset):
    def __init__(self, data_list, scores_list, transform=None, pre_transform=None):
        super().__init__(".", transform, pre_transform)
        self.data = data_list
        for X, score in zip(self.data, scores_list):
            X.y = torch.Tensor(score)

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return []

    def len(self):
        return len(self.data)

    def get(self, idx):
        return self.data[idx]

    def process(self):
        pass


# Create the dataset
train_dataset = MyDataset(X_train, scores_list[: len(X_train)])
test_dataset = MyDataset(X_test, scores_list[len(X_train) :])

In [ ]:
from sklearn.model_selection import GridSearchCV

from mqt.predictor.ml import GNNClassifier

color1 = "#21918c"
color2 = "#440154"

clf = GNNClassifier(
    optimizer="adam",
    learning_rate=1e-3,
    batch_size=32,
    epochs=10,
    num_node_categories=42,  # distinct gate types (incl. 'id' and 'meas')
    num_edge_categories=2,  # is_control. or 100,  #for  distinct wires (quantum + classical)
    node_embedding_dim=4,  # dimension of the node embedding
    edge_embedding_dim=4,  # dimension of the edge embedding
    num_layers=4,  # number of nei aggregations
    hidden_dim=16,  # dimension of the hidden layers
    output_dim=7,  # dimension of the output vector
)

param_grid = {
    "optimizer": ["adam"],
    "learning_rate": [1e-3],
    "batch_size": [32],
    "epochs": [100],
    "hidden_dim": [16],
}

clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=1).fit(train_dataset)

y_pred = np.array(list(clf.predict(test_dataset)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="MLPClassifier", color=color1)
rel_goodness = np.round(np.mean(rel_scores), 4)
rel_goodness_std = np.round(np.std(rel_scores), 4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Multilayer Perceptron", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# Data Preparation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm, tree
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from mqt.predictor import ml

np.random.seed(10)

predictor = ml.Predictor()
figure_of_merit = "expected_fidelity"

training_data = predictor.get_prepared_training_data(figure_of_merit=figure_of_merit, save_non_zero_indices=True)

X_train = training_data.X_train
X_test = training_data.X_test
y_train = training_data.y_train
y_test = training_data.y_test
indices_train = training_data.indices_train
indices_test = training_data.indices_test
names_list = training_data.names_list
scores_list = training_data.scores_list

scores_filtered = [scores_list[i] for i in indices_test]
names_filtered = [names_list[i] for i in indices_test]

performance = []

In [ ]:
color1 = "#21918c"
color2 = "#440154"

In [ ]:
print(X_train)

In [ ]:
print(len(X_train), len(X_test))

# Random Forest

In [ ]:
clf = RandomForestClassifier(random_state=0)
tree_param = [
    {
        "n_estimators": [100, 200, 500],
        "max_depth": list(range(8, 30, 6)),
        "min_samples_split": list(range(2, 20, 6)),
        "min_samples_leaf": list(range(2, 20, 6)),
        "bootstrap": [True, False],
    },
]
clf = GridSearchCV(clf, tree_param, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="RandomForestClassifier", color=color1)
rel_goodness = np.round(np.mean(rel_scores), 4)
rel_goodness_std = np.round(np.std(rel_scores), 4)
print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Feature Importance: ", clf.best_estimator_.feature_importances_)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Random Forest", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

In [ ]:
predictor.plot_eval_all_detailed_compact_normed(
    names_filtered, scores_filtered, y_pred, y_test, color_all=color1, color_pred=color2
)

In [ ]:
clf.best_estimator_.get_params()

### Feature Importances

In [ ]:
path = ml.helper.get_path_trained_model(figure_of_merit, return_non_zero_indices=True)
non_zero_indices = np.load(str(path), allow_pickle=True)

openqasm_qc_list = ml.helper.get_openqasm_gates()
feature_names = [openqasm_qc_list[i] for i in range(len(openqasm_qc_list))]
feature_names.append("num qubits")
feature_names.append("depth")
feature_names.append("prog. comm.")
feature_names.append("crit. dept")
feature_names.append("entang. ratio")
feature_names.append("parallelism")
feature_names.append("liveness")
feature_names = [feature_names[i] for i in non_zero_indices]

importances = clf.best_estimator_.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.best_estimator_.estimators_], axis=0)

idx = np.argsort(-importances)

plt.figure(figsize=(8, 6))
plt.bar(np.array(feature_names)[idx], np.array(importances)[idx], color=color1, width=0.9)
plt.errorbar(
    np.array(feature_names)[idx],
    np.array(importances)[idx],
    np.array(std)[idx],
    fmt="o",
    color=color2,
)
plt.xticks(rotation=90, fontsize=18)
plt.yticks(fontsize=18)
plt.ylabel("Relative feature importance", fontsize=18)
plt.tight_layout()
plt.savefig("results/feature_importances.pdf")
plt.show()

#### Check the relative importances per feature

In [ ]:
summary = zip(np.array(feature_names)[idx], np.array(importances)[idx])
for feature, importance in list(summary):
    print(feature, np.round(importance, 3))

# GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier()

param_grid = {
    "learning_rate": [0.01, 0.1, 1],
}

clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))

res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="GradientBoostingClassifier", color=color1)
rel_goodness = np.round(np.mean(rel_scores), 4)
rel_goodness_std = np.round(np.std(rel_scores), 4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Gradient Boosting", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# Decision Tree Classifier

In [ ]:
clf = tree.DecisionTreeClassifier(random_state=5)

tree_param = [
    {
        "criterion": ["entropy", "gini"],
        "max_depth": list(range(1, 15, 1)),
        "min_samples_split": list(range(2, 20, 4)),
        "min_samples_leaf": list(range(2, 20, 4)),
        "max_leaf_nodes": list(range(2, 200, 40)),
        "max_features": list(range(1, len(non_zero_indices), 10)),
    },
]
clf = GridSearchCV(clf, tree_param, cv=5, n_jobs=8).fit(X_train, y_train)
y_pred = np.array(list(clf.predict(X_test)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="DecisionTreeClassifier", color=color1)
rel_goodness = np.round(np.mean(rel_scores), 4)
rel_goodness_std = np.round(np.std(rel_scores), 4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
print("Feature Importance: ", clf.best_estimator_.feature_importances_)
performance.append(("Decision Tree", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# Nearest Neighbor

In [ ]:
clf = KNeighborsClassifier()
param_grid = {"n_neighbors": range(1, 10, 1)}
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="KNeighborsClassifier", color=color1)
rel_goodness = np.round(np.mean(rel_scores), 4)
rel_goodness_std = np.round(np.std(rel_scores), 4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Nearest Neighbor", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# MLPClassifier

In [ ]:
clf = MLPClassifier(max_iter=1000)

param_grid = {
    "hidden_layer_sizes": [(50, 50, 50), (50, 100, 50), (100,)],
    "activation": ["tanh", "relu"],
    "solver": ["sgd", "adam"],
    "alpha": [0.0001, 0.05],
    "learning_rate": ["constant", "adaptive"],
}

clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="MLPClassifier", color=color1)
rel_goodness = np.round(np.mean(rel_scores), 4)
rel_goodness_std = np.round(np.std(rel_scores), 4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Multilayer Perceptron", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# SVM

In [ ]:
clf = svm.SVC()
param_grid = {"C": [0.1, 1, 10], "gamma": [1, 0.1, 0.01], "kernel": ["rbf", "sigmoid"]}
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="SVM", color=color1)
rel_goodness = np.round(np.mean(rel_scores), 4)
rel_goodness_std = np.round(np.std(rel_scores), 4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Support Vector Machine", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# Naive Bayes

In [ ]:
clf = GaussianNB()
param_grid = {"var_smoothing": np.logspace(0, -9, num=100)}
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="GaussianNB", color=color1)
rel_goodness = np.round(np.mean(rel_scores), 4)
rel_goodness_std = np.round(np.std(rel_scores), 4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Naive Bayes", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# Save Performance Results

In [ ]:
print(performance)
from pathlib import Path

file = Path("results/performances_" + figure_of_merit + ".csv")
with file.open("w") as f:
    f.write("Classifier, Accuracy, Top3, Worst Rank, Eval. Score Diff., Std\n")
    for sublist in performance:
        line = f"{sublist[0]}, {sublist[1]}, {sublist[2]}, {sublist[3]}, {sublist[4]}, {sublist[5]} \n"
        f.write(line)